In [12]:
## CSE 601 :Data Mining and Bio-informatics
## Swathi Ravindran 
## Akshara Santharam
## Sai Kumar Aindla

#Frequent itemsets function
def frequent_set(data, support = 50):
    support = support / 100
    Count = 1  
    Temporary, final = item_set(data, support)
    final_data = []
    for r in data:
        tmp = set()
        for c in r:
            tmp.add(c)
        final_data.append(tmp)

    while (1):
        Count = Count + 1
        set_l = set()
        i = 0
        while i < len(Temporary):
            j = 0
            while j < len(Temporary):
                temp_set = Temporary[i] | Temporary[j]

                temp_set = sorted(temp_set)

                if len(temp_set) == Count:

                    set_l.add(frozenset(temp_set))
                j += 1
            i += 1


        Temporary = []                    

        for var1 in set_l:

            number = 0
            for i in final_data:

                if var1.issubset(i):

                    number = number + 1

            if support <= number / len(data):

                final[var1] = number
                Temporary.append(var1)

        if len(Temporary) != 0:
            print("Number of length {} frequent itemsets {}".format(Count,len(Temporary)))
        else:
            break

    return final

#Number of Itemsets
def item_set(data, support):
    Arr = {}
    final_data = {}

    for row in range(len(data)):
        for col in range(len(data[row])):
            Arr[data[row][col]]  = (Arr[data[row][col]] + 1) if (data[row][col] in Arr) else 1

    number = 0
    Temporary = []

    for item in Arr:
        if support <= Arr[item] / len(data):

            temp_set = set()

            temp_set.add(item)

            final_data[frozenset(temp_set)] = Arr[item]  

            Temporary.append(temp_set)

            number = number + 1

    #Outputing the lenght 1 itemsets
    print("Number of length {} frequent itemsets {}".format(1,len(Temporary)))

    return Temporary, final_data

# Finding Minimum confidence         
def minimum_confidence(r, f_items, confidence):
    x = r[0] | r[1]
    support_count = float(f_items[x])
    if confidence <= (support_count / float(f_items[r[0]])):
        return True   
    return False
    
#Remove Duplicates
def remove_dups(dups):
    final_set = set(tuple(res) for res in dups)
    return [list(res) for res in final_set]


#Association Class for finding rule and defining Templates
class Association_Rule:
    def __init__(self, support, confidence, f_items):
        self.support = support
        self.confidence = confidence
        self.freqItemsets = f_items
        self.rules = []
        for Item_set_frequent in self.freqItemsets:
            if len(Item_set_frequent) > 1:
                self.Rules(Item_set_frequent)
                    
# -------------------------------------- Finding rules ---------------------------------------------------------------
    def Rules(self, Item_set_frequent):
        Previous = []
        frequent_length = len(Item_set_frequent)
        
        for item in Item_set_frequent:
            
            temp_items = set(Item_set_frequent)
            
            temp_items.remove(item)
            
            temp_var = [frozenset(temp_items), frozenset({item})]
            
            if minimum_confidence(temp_var, self.freqItemsets, self.confidence ):
                
                Previous.append(temp_var)
                self.rules.append(temp_var)         
        
        while frequent_length > 2:
            
            combinations = itertools.combinations(Previous, 2)
            temp_var = []

            for comb in combinations:
                
                x = comb[0][1] | comb[1][1]
                i = comb[1][0] & comb[0][0]
                
                if len(i) == frequent_length - 2:
                    temp_list = [frozenset(set(i)), frozenset(set(x))]
                    
                    if minimum_confidence(temp_list, self.freqItemsets, self.confidence):
                        temp_var.append(temp_list)

            self.rules = self.rules + temp_var
            Previous = temp_var
            frequent_length = frequent_length - 1

#   ---------------------------------------------- Template 1---------------------------------------------------------------   
    def Template1(self, r, apriori_condition, count, items):
        
        if count == "ANY" and apriori_condition == "RULE":
            if len(set(items) & (r[0] | r[1])) > 0:
                return True
            
        elif isinstance(count, int) and apriori_condition == "RULE":
            if len(set(items) & (r[0] | r[1])) == count:
                return True
            
        elif apriori_condition == "RULE" and count == "NONE":
            if len(set(items) & (r[0] | r[1]) ) == 0:
                return True
       
        elif apriori_condition == "HEAD" and count == "NONE":
            if len(set(items) & r[0]  ) == 0:
                return True
            
        elif apriori_condition == "BODY" and count == "ANY":
            if len(set(items) & r[1]  ) > 0:
                return True
            
        elif apriori_condition == "HEAD" and count == "ANY":
            if len(set(items) & r[0] ) > 0:
                return True
         
        elif apriori_condition == "BODY" and isinstance(count, int):
            if len(set(items) & r[1] ) == count:
                return True
            
        elif apriori_condition == "HEAD" and isinstance(count, int):
            if len(set(items) & r[0] ) == count:
                return True
        
        elif apriori_condition == "BODY" and count == "NONE":
            if len(set(items) & r[1] ) == 0:
                return True
       
        return False
    

    def Temp_1(self, apriori_condition, count, items):
        output = []
        for r in self.rules:
            if self.Template1(r, apriori_condition, count, items):
                output.append(r)
        output = remove_dups(output)
        return output, len(output)
# ----------------------------------------------- Template 2---------------------------------------------------------------
    
    def Template2(self, r, apriori_condition, count):
        
        if len(r[1]) >= count and apriori_condition == "BODY":
            return True
        
        elif (len(r[0]) + len(r[1])) >= count and apriori_condition == "RULE":
            return True
        
        elif len(r[0]) >= count and apriori_condition == "HEAD":
            return True
        
        return False   

    def Temp_2(self, apriori_condition, count):
        
        output = []
        
        for r in self.rules:
            
            if self.Template2(r, apriori_condition, count):
                output.append(r)
                
        output = remove_dups(output)
        return output, len(output)
# -------------------------------------------------------- Template 3------------------------------------------------------  
    
    def Template3(self, r, args):
        
       
        check = args[0]
        
        if check[-1] == '1' and check[0] == '1':
            
            output_2 = self.Template1(r, args[4], args[5], args[6])
            output_1 = self.Template1(r, args[1], args[2], args[3])
            
        elif check[-1] == '1' and check[0] == '2':
            
            output_2 = self.Template1(r, args[3], args[4], args[5])
            output_1 = self.Template2(r, args[1], args[2])
            
        elif check[-1] == '2'and check[0] == '2':
            
            output_2 = self.Template2(r, args[3], args[4])
            output_1 = self.Template2(r, args[1], args[2])
            
        elif check[-1] == '2'and check[0] == '1':
            
            output_2 = self.Template2(r, args[4], args[5])
            output_1 = self.Template1(r, args[1], args[2], args[3])
        
        if check[1:-1] == 'or':
            return output_1 or output_2    

        elif check[1:-1] == 'and': 
            return output_1 and output_2

    def Temp_3(self, *args):
        
        output = []
        for r in self.rules:
            
            if self.Template3(r, args):
                output.append(r)
                
        output = remove_dups(output)
        return output, len(output)

In [13]:
import itertools
#Main Function

support = 50
confidence= 70

file = open("association-rule-test-data.txt", "r")
data = [line.strip().split('\t') for line in file.readlines()]
for x in range(len(data)):
    counter=1
    for y in range(len(data[x])):
            data[x][y] = 'G' + str(counter) + '_' + data[x][y]
            counter+=1      

#Number of frequent Itemsets
freq_item_sets = frequent_set(data,support)
print("Total Frequent Itemset Generated for Support {} and Confidence {} : {}".format(support,confidence,str(len(freq_item_sets))))
#Association Rules
asso_rule = Association_Rule(float(support), float(confidence)/100,freq_item_sets)



Number of length 1 frequent itemsets 109
Number of length 2 frequent itemsets 63
Number of length 3 frequent itemsets 2
Total Frequent Itemset Generated for Support 50 and Confidence 70 : 174


In [14]:
#### TEMPLATE 1 Results ####

In [15]:
(result11, cnt) = asso_rule.Temp_1("RULE", "ANY", ['G59_Up'])

In [16]:
cnt

26

In [17]:
(result12, cnt) = asso_rule.Temp_1("RULE", "NONE", ['G59_Up'])

In [18]:
cnt

91

In [19]:
(result13, cnt) = asso_rule.Temp_1("RULE", 1, ['G59_Up', 'G10_Down']) 

In [20]:
cnt

39

In [21]:
(result14, cnt) = asso_rule.Temp_1("HEAD", "ANY", ['G59_Up'])

In [22]:
cnt

9

In [23]:
 (result15, cnt) = asso_rule.Temp_1("HEAD", "NONE", ['G59_Up']) 

In [24]:
cnt

108

In [25]:

(result16, cnt) = asso_rule.Temp_1("HEAD", 1, ['G59_Up', 'G10_Down']) 

In [26]:
cnt

17

In [27]:
(result17, cnt) = asso_rule.Temp_1("BODY", "ANY", ['G59_Up']) 

In [28]:
cnt

17

In [29]:
(result18, cnt) = asso_rule.Temp_1("BODY", "NONE", ['G59_Up']) 

In [30]:
cnt

100

In [31]:
(result19, cnt) = asso_rule.Temp_1("BODY", 1, ['G59_Up', 'G10_Down'])

In [32]:
cnt

24

In [33]:
###### TEMPLATE 2 Results ####

In [34]:
(result21, cnt) = asso_rule.Temp_2("RULE", 3) 

In [35]:
cnt

9

In [36]:
(result22, cnt) = asso_rule.Temp_2("HEAD", 2) 

In [37]:
cnt

6

In [38]:
(result23, cnt) = asso_rule.Temp_2("BODY", 1)

In [39]:
cnt

117

In [40]:
#### TEMPLATE 3 Results ####

In [41]:
(result31, cnt) = asso_rule.Temp_3("1or1", "HEAD", "ANY", ['G10_Down'], "BODY", 1, ['G59_Up']) 

In [42]:
cnt

24

In [43]:
(result32, cnt) = asso_rule.Temp_3("1and1", "HEAD", "ANY", ['G10_Down'], "BODY", 1, ['G59_Up'])

In [44]:
cnt

1

In [45]:
(result33, cnt) = asso_rule.Temp_3("1or2", "HEAD", "ANY", ['G10_Down'], "BODY", 2) 

In [46]:
cnt

11

In [47]:
(result34, cnt) = asso_rule.Temp_3("1and2", "HEAD", "ANY", ['G10_Down'], "BODY", 2) 

In [48]:
cnt

0

In [49]:
(result35, cnt) = asso_rule.Temp_3("2or2", "HEAD", 1, "BODY", 2) 

In [50]:
cnt

117

In [51]:
(result36, cnt) = asso_rule.Temp_3("2and2", "HEAD", 1, "BODY", 2) 

In [52]:
cnt

3

In [53]:
(result, cnt) = asso_rule.Temp_1("HEAD", "NONE", ['G1_UP'])
cnt

117

In [54]:
(result, cnt) = asso_rule.Temp_1("HEAD", 1, ['G1_UP', 'G10_Down'])
cnt

8

In [55]:
(result, cnt) = asso_rule.Temp_2("RULE", 2)
cnt

117

In [56]:
(result, cnt) = asso_rule.Temp_2("BODY", 2)
cnt

3

In [57]:
(result, cnt) = asso_rule.Temp_3("1or2", "HEAD", "ANY", ['G1_UP'], "BODY", 2)
cnt

3

In [58]:
(result, cnt) = asso_rule.Temp_3("1and2", "HEAD", "ANY", ['G1_UP'], "BODY", 2)
cnt

0

In [59]:
(result, cnt) = asso_rule.Temp_3("2and2", "HEAD", 1, "BODY", 2)
result, cnt

([[frozenset({'G82_Down'}), frozenset({'G59_Up', 'G72_Up'})],
  [frozenset({'G96_Down'}), frozenset({'G59_Up', 'G72_Up'})],
  [frozenset({'G72_Up'}), frozenset({'G59_Up', 'G82_Down'})]],
 3)